In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

model_name = "searle-j/kote_for_easygoing_people"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(model=model,
                                  tokenizer=tokenizer,
                                  device=-1,
                                  return_all_scores=True,
                                  function_to_apply='sigmoid')

sentences = [
    "아침에 일어나긴 했는데, 몸이 좀 무거워서 한참을 가만히 있었어요.",
    "해야 할 건 했죠. 밥도 먹고 설거지도 했는데, 예전처럼 손이 잘 안 가더라고요.",
    "날씨는 괜찮던데, 그냥 집에 있는 게 더 편해서요.",
    "네, 괜히 나갔다가 피곤해질까 봐요. 집에 있으면 조용하잖아요.",
    "괜찮다고는 하는데, 사실 하루가 그냥 흘러가는 느낌이에요."
]

with open('results.txt', 'w', encoding='utf-8') as f:
    for i, sentence in enumerate(sentences, 1):
        f.write(f"\n[{i}] 입력 문장: {sentence}\n")
        f.write("=" * 70 + "\n")
        results = pipe(sentence)[0]
        for output in results:
            if output["score"] > 0.4:
                f.write(f"  - {output['label']}: {output['score']:.4f}\n")
        f.write("\n")
    
    f.write("완료!\n")

print("results.txt 파일로 저장되었습니다.")


Device set to use cpu
c:\Users\USER\Documents\GitHub\KOTE\.venv\lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


results.txt 파일로 저장되었습니다.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import pandas as pd
from tqdm import tqdm

model_name = "searle-j/kote_for_easygoing_people"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(model=model,
                                  tokenizer=tokenizer,
                                  device=0,
                                  return_all_scores=True,
                                  function_to_apply='sigmoid')

# 감정 분류 매핑
emotion_mapping = {
    # 긍정 (15개)
    '환영/호의': '긍정',
    '감동/감탄': '긍정',
    '고마움': '긍정',
    '존경': '긍정',
    '기대감': '긍정',
    '뿌듯함': '긍정',
    '편안/쾌적': '긍정',
    '신기함/관심': '긍정',
    '아껴주는': '긍정',
    '흐뭇함(귀여움/예쁨)': '긍정',
    '행복': '긍정',
    '안심/신뢰': '긍정',
    '깨달음': '긍정',
    '즐거움/신남': '긍정',
    '기쁨': '긍정',
    
    # 중립 (1개)
    '없음': '중립',
    
    # 부정 (20개)
    '불평/불만': '부정',
    '지긋지긋': '부정',
    '화남/분노': '부정',
    '의심/불신': '부정',
    '부끄러움': '부정',
    '공포/무서움': '부정',
    '절망': '부정',
    '한심함': '부정',
    '역겨움/징그러움': '부정',
    '짜증': '부정',
    '어이없음': '부정',
    '패배/자기혐오': '부정',
    '증오/혐오': '부정',
    '당황/난처': '부정',
    '경악': '부정',
    '놀람': '부정',
    '불안/걱정': '부정',
    '우쭐댐/무시함': '부정',
    '비장함': '부정',
    
    # 활력 저하 (8개)
    '슬픔': '활력 저하',
    '안타까움/실망': '활력 저하',
    '서러움': '활력 저하',
    '힘듦/지침': '활력 저하',
    '재미없음': '활력 저하',
    '부담/안_내킴': '활력 저하',
    '불쌍함/연민': '활력 저하',
    '귀찮음': '활력 저하',
    '죄책감': '부정',
}

# 각 분류별 감정 개수
category_counts = {
    '긍정': 15,
    '중립': 1,
    '부정': 20,
    '활력 저하': 8,
}

def process_file(file_path, file_type):
    """파일을 읽고 감정 분석 수행"""
    df = pd.read_csv(file_path, sep='\t', header=None, names=['id', 'text', 'labels'])
    results = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"{file_type} 분석 진행 중"):
        text = row['text']
        outputs = pipe(text)[0]
        
        # 감정 분류별 스코어 합계
        emotion_scores = {'긍정': 0, '중립': 0, '활력 저하': 0, '부정': 0}
        
        for output in outputs:
            if output["score"] > 0:
                label = output['label']
                score = output['score']
                
                # 라벨을 분류로 매핑
                if label in emotion_mapping:
                    category = emotion_mapping[label]
                    emotion_scores[category] += score
        
        # 각 분류별 감정 개수로 나누기
        positive = emotion_scores['긍정'] / category_counts['긍정']
        neutral = emotion_scores['중립'] / category_counts['중립']
        vitality = emotion_scores['활력 저하'] / category_counts['활력 저하']
        negative = emotion_scores['부정'] / category_counts['부정']
        
        result = {
            '긍정': round(positive, 4),
            '중립': round(neutral, 4),
            '활력 저하': round(vitality, 4),
            '부정': round(negative, 4),
            'Y': round(positive * 1 + neutral * 0 + vitality * 1.5 + negative * 2, 4),
        }
        results.append(result)
    
    return pd.DataFrame(results)

# 3개 파일 처리
train_df = process_file('train.tsv', 'train')
val_df = process_file('val.tsv', 'val')
test_df = process_file('test.tsv', 'test')

# 모든 데이터 합치기
final_df = pd.concat([train_df, val_df, test_df], ignore_index=True)

# 출력
print(final_df)
print(f"\n총 {len(final_df)}개 행 처리 완료")

# CSV로 저장
final_df.to_csv('emotion_classification_final.csv', index=False, encoding='utf-8-sig')
print("결과가 'emotion_classification_final.csv'로 저장되었습니다.")

In [ ]:
import pandas as pd

# 저장된 파일 읽기
final_df = pd.read_csv('emotion_classification_final.csv', encoding='utf-8-sig')

# 필요한 컬럼만 선택
result_df = final_df[['긍정', '중립', '활력 저하', '부정', 'Y']]

# 출력
print(result_df)

# CSV로 저장
result_df.to_csv('final_cleaned.csv', index=False, encoding='utf-8-sig')
print("결과가 'emotion_classification_final_cleaned.csv'로 저장되었습니다.")
